# Mini Project 1

**2025 Introduction to Quantiative Methods in Finance**

**The Erdös Institute**

**Instructions** Use current stock data to create two potentially profitable investment portfolios. One that is higher risk and one that is lower risk.

-- You are to interpret and explain your interpretation of a high risk profile and low risk profile of a portfolio. You should provide some measurable quantitative data in your explanation.

In this poject, we will maximize the objective function $f(weights) = \textrm{profit}(weights) - \lambda\cdot\textrm{volatility}(weights)$ with different $\lambda$ constants $(\lambda = 1, \lambda = 7)$.

By increasing the lambda, we will have higher risk portfolio with a possibly higher profit.

In [51]:
### Import modules ###

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.optimize import minimize
import datetime as dt
sns.set_style('darkgrid')

import yfinance as yf

In [52]:
### Load data for each stock from yfinance ###
tickers = ['MSFT', 'GOOGL', 'AMZN', 'TSLA', 'F']

#yf.Ticker(tickers[5]).info

start_date = dt.datetime.today() - dt.timedelta(days = 1*365)
end_date = dt.datetime.today() - dt.timedelta(days = 0*365)

stock = yf.download(tickers, start = start_date, end =end_date)

/tmp/ipython-input-52-22305822.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock = yf.download(tickers, start = start_date, end =end_date)
[*********************100%***********************]  5 of 5 completed


In [54]:
### Setting parameters and constants ###

#Calculate the covariance matrix
daily_returns = np.log(stock['Close']/stock['Close'].shift(1))
daily_returns = daily_returns.dropna()
covariance_matrix = 252*((daily_returns).cov())


#Create a dictionary of standard deviations of daily return
#Normalize the standard deviation for yearly by multiplying by sqrt(#trading days in year ~ 252)
annualized_volatility = {ticker: np.std(daily_returns[ticker])*np.sqrt(252) for ticker in tickers}



# Number of assets
n_assets = len(tickers)

# Define an initial guess for asset weights (n=5 in our case)
initial_weights = np.array([1/n_assets] * n_assets)

# Define weight constraints
#Sum of weights equals 1
#Allocate at least 5% of capital into each index in stock_symbols
#Do not allocate more than 35% of capital into each index in stock_symbol

constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights)-1},
               {'type': 'ineq', 'fun': lambda weights: min(weights)-.05},
              {'type': 'ineq', 'fun': lambda weights: .35-max(weights)})



In [58]:
### Define functions ###

def annual_log_return(weights):
    portfolio_value = (stock['Close'] * weights).sum(axis=1)
    daily_log_returns = np.log(portfolio_value / portfolio_value.shift(1))
    annualized_log_return = np.sum(daily_log_returns.dropna())
    return annualized_log_return

def portfolio_volatility(weights):
    portfolio_std_dev = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))
    return portfolio_std_dev

def objective_1(weights):
    return -(annual_log_return(weights) - 7.0*portfolio_volatility(weights)) # lambda = 7

def objective_2(weights):
    return -(annual_log_return(weights) - 1.0*portfolio_volatility(weights)) # lambda = 1



In [59]:
### Simulate and print the results ###

result_1 = minimize(objective_1, initial_weights, constraints=constraints)
result_2 = minimize(objective_2, initial_weights, constraints=constraints)



# Optimal asset weights

optimal_weights_1 = result_1.x
optimal_weights_2 = result_2.x


#print(result_1)

# Print the optimal weights and expected returns
print("Optimal Asset Weights to minimize ojective_1:")
for ticker, weight in zip(tickers, optimal_weights_1):
    print(f"{ticker}: Weight = {weight:.4f}")

print('----'*20)

print("Optimal Asset Weights to minimize objective_2:")
for ticker, weight in zip(tickers, optimal_weights_2):
    print(f"{ticker}: Weight = {weight:.4f}")

print('----'*20)

print("Annual log profit for objective_1:")
print(annual_log_return(optimal_weights_1))
print("Volatility for ojective_1:")
print(portfolio_volatility(result_1.x))

print('----'*20)

print("Annual log profit for objective_2:")
print(annual_log_return(optimal_weights_2))
print("Volatility for ojective_2:")
print(portfolio_volatility(result_2.x))

/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Optimal Asset Weights to minimize ojective_1:
MSFT: Weight = 0.0857
GOOGL: Weight = 0.2685
AMZN: Weight = 0.2458
TSLA: Weight = 0.3500
F: Weight = 0.0500
--------------------------------------------------------------------------------
Optimal Asset Weights to minimize objective_2:
MSFT: Weight = 0.2000
GOOGL: Weight = 0.3500
AMZN: Weight = 0.0500
TSLA: Weight = 0.0500
F: Weight = 0.3500
--------------------------------------------------------------------------------
Annual log profit for objective_1:
0.09039249430010711
Volatility for ojective_1:
0.25530729881536096
--------------------------------------------------------------------------------
Annual log profit for objective_2:
0.2994312911390325
Volatility for ojective_2:
0.3893462271597941


As we can see in the result, the objective_2 function has higher volatility compared to objective_1 function, because we chose larger $\lambda$ (hence more importance in minimizing volatility in the objective function).

However, the annual log profit is larger for objective_1 because we are putting less importance in minmizing volatility and more importance in maximizing the profit.